Dataset overview

```
{
    "sections": [
    {
      "metadata": {
       "section_name": "Recruitment Policy",
       "keywords": ["manpower requisition", "recruitment process", "interview panel", "medical examination"],
       "page_number": [9, 11],
       "category": "Recruitment",
       "sub_category": "Recruitment Process",
       "difficulty_level": "Intermediate",
       "date_updated": "2024-09-10",
       "qa_pairs": [
         {
         "input": "What is the process for raising a manpower requisition at IIMA?",
         "response": "Manpower requisition at IIMA is initiated by the concerned department by filling a Manpower Requisition Form (MRF). The MRF must include the pay level, tenure of employment, and a brief job description. The requisition is approved by the Director before recruitment begins.",
         "source": "HR Policy Manual, Page 9"
         },
         {
         "input": "What documents are required during the joining process at IIMA?",
         "response": "During the joining process, candidates are required to submit documents including mark sheets from 10th to the last qualified exam, passing certificates, birth certificate, address proof, relieving letter from the last employer, medical fitness certificate, and identification proof such as Aadhar card, PAN card, or passport.",
         "source": "HR Policy Manual, Page 11"
         },
         {
         "input": "Who constitutes the interview panel for positions at IIMA?",
         "response": "For Manager and above positions, the interview panel includes members such as the OEDC member, Dean, Chief Administrative Officer, Head of Department, and the Associate Vice President - HR. Additional experts may be included if required.",
         "source": "HR Policy Manual, Page 10"
         },
         {
         "input": "What are the criteria for selecting a candidate at IIMA?",
         "response": "Candidates are selected based on personal interviews, skill tests, group discussions, and reference checks. The final selection is subject to a medical examination.",
         "source": "HR Policy Manual, Page 11"
         },
         {
         "input": "What is the role of the HR department in recruitment at IIMA?",
         "response": "The HR department is responsible for processing applications, scheduling interviews, conducting background checks, and negotiating salaries during the recruitment process.",
         "source": "HR Policy Manual, Page 9"
         },
         {
         "input": "What is the medical examination requirement during recruitment?",
         "response": "All candidates must undergo a medical examination either at the IIMA facility or through a recognized civil hospital. The doctor may request additional tests if necessary.",
         "source": "HR Policy Manual, Page 11"
         },
         {
         "input": "What steps are included in IIMA's joining process?",
         "response": "The joining process includes the submission of the Candidate’s Statement & Declaration, document verification, submission of a joining report, and the issuing of a Joining Memorandum.",
         "source": "HR Policy Manual, Page 11"
         },
         {
         "input": "What kind of advertisement process is followed for recruitment at IIMA?",
         "response": "IIMA releases recruitment advertisements in newspapers, and feedback is gathered from the relevant department heads before publication.",
         "source": "HR Policy Manual, Page 9"
         },
         {
         "input": "How are applications processed during recruitment?",
         "response": "The HR department is responsible for segregating resumes, compiling the details of applicants, and forwarding them to the concerned department for shortlisting.",
         "source": "HR Policy Manual, Page 9"
         },
         {
         "input": "What kind of travel reimbursement is provided to candidates during recruitment?",
         "response": "For Manager and above positions, economy air travel fare is reimbursed. For Level 6 to Level 9 positions, AC 3-Tier train fare is reimbursed, and Sleeper Class train fare is reimbursed for lower levels.",
         "source": "HR Policy Manual, Page 10"
         }
        ]
      },
      "description": "The recruitment process at IIMA starts with the Manpower Requisition Form (MRF). This form is filled by the concerned department, detailing the pay level, tenure of employment, job description, and justification for the position. The approval process requires the MRF to be signed off by the department chairperson and then forwarded to the HR department. The HR Manager and the Associate Vice President-HR discuss the proposal with the department head and the Director, who give the final approval. Once approved, the job advertisement is prepared and released in selected newspapers. The advertisement copy may also be reviewed by the Head of Department for feedback before publication. The HR department segregates resumes based on eligibility and sends them to the concerned department for shortlisting. After shortlisting, the interview panel is constituted. For Manager and above positions, the interview panel includes the OEDC member, Dean, Chief Administrative Officer, Head of Department, Associate Vice President-HR, and any expert nominated by the Director. For positions below Manager level, the panel consists of the SEDC member, Head of Department, Associate VP-HR, and a nominated person. The interview process may include personal interviews, skill tests, group discussions, debates, or quizzes. Candidates are informed via email, followed by a formal letter for interview scheduling. After the interview, feedback from panel members is documented, and the HR department assists with salary negotiations and collects feedback from references if necessary. Travel reimbursement for Manager and above positions covers economy class airfare. For candidates applying to positions from Level 6 to Level 9, AC 3-tier train fare is reimbursed, while those below Level 6 are reimbursed sleeper class train fare. After selection, candidates must undergo a medical check-up or submit a fitness certificate. The joining procedure involves collecting primary details through a Candidate’s Statement and Declaration form and verifying documents such as mark sheets, certificates, birth certificate, address proof, relieving letter, latest salary slip, medical certificate, and identity proof. The final steps include the issuance of a joining report and an identity card."
  
    },

```

In [3]:
%pip install -qU langchain langchain-faiss faiss-cpu sentence-transformers

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchain-faiss (from versions: none)
ERROR: No matching distribution found for langchain-faiss


In [1]:
import json
from langchain.schema import Document

# FAISS imports
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS

def split_into_batches(docs, batch_size):
    for i in range(0, len(docs), batch_size):
        yield docs[i:i + batch_size]


def load_and_process_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)

    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

    Full_HR = []
    QA_HR = []

    # Iterate over each section in the JSON data
    for section in data.get('sections', []):
        metadata = section.get('metadata', {})
        description = section.get('description', '')
    
        # Extract metadata fields
        section_name = metadata.get('section_name', '')
        keywords = metadata.get('keywords', [])
        page_number = metadata.get('page_number', [])
        category = metadata.get('category', '')
        sub_category = metadata.get('sub_category', '')
        difficulty_level = metadata.get('difficulty_level', '')
        date_updated = metadata.get('date_updated', '')
        qa_pairs = metadata.get('qa_pairs', [])
    
        # Create a Document for the section description
        doc_metadata = {
            'section_name': section_name,
            'keywords': ', '.join(keywords) if isinstance(keywords, list) else keywords,
            'page_number': ', '.join(map(str, page_number)) if isinstance(page_number, list) else page_number,
            'category': category,
            'sub_category': sub_category,
            'difficulty_level': difficulty_level,
            'date_updated': date_updated,
        }
    
        description_doc = Document(
            page_content=description,
            metadata=doc_metadata
        )
        Full_HR.append(description_doc)
    
        # Create Documents for each QA pair
        for index, qa in enumerate(qa_pairs):
            question = qa.get('input', '')
            answer = qa.get('response', '')
            source = qa.get('source', '')
            qa_content = f"Question: {question}\nAnswer: {answer}\nSource: {source}"
            qa_metadata = {
                'section_name': section_name,
                'keywords': ', '.join(keywords) if isinstance(keywords, list) else keywords,
                'page_number': ', '.join(map(str, page_number)) if isinstance(page_number, list) else page_number,
                'category': category,
                'sub_category': sub_category,
                'difficulty_level': difficulty_level,
                'date_updated': date_updated,
                'source': source
            }
    
            QA_doc = Document(
                page_content=qa_content,
                metadata=qa_metadata
            )
            QA_HR.append(QA_doc)

    # Create FAISS vector stores
    faiss_full_hr = FAISS.from_documents(Full_HR, embedding_function)
    faiss_qa_hr = FAISS.from_documents(QA_HR, embedding_function)

    # Optionally, save the FAISS indexes for later use
    faiss_full_hr.save_local("HR/Vector/Full_HR")
    faiss_qa_hr.save_local("HR/Vector/QA_HR")

    return faiss_full_hr, faiss_qa_hr


# Path to your JSON file
file_path = 'HR/hr-policy-final.json'
db_Full_HR, db_QA_HR = load_and_process_data(file_path)


C:\Users\priya\AppData\Local\Temp\ipykernel_29396\3545694070.py:17: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\priya\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


C:\Users\priya\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\priya\AppData\Roaming\Python\Python311\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [1]:
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Load the FAISS indexes
faiss_full_hr = FAISS.load_local("HR/Vector/Full_HR", embedding_function,allow_dangerous_deserialization=True)
faiss_qa_hr = FAISS.load_local("HR/Vector/QA_HR", embedding_function,allow_dangerous_deserialization=True)


C:\Users\priya\AppData\Local\Temp\ipykernel_27068\919284486.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\priya\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


C:\Users\priya\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Example query
query = "What is the process for recruitment at IIMA?"

# Perform similarity search on the QA vector store
docs = faiss_full_hr.similarity_search(query, k=20)

# Display the results
for doc in docs:
    print(f"Content: {doc.page_content}\nMetadata: {doc.metadata}\n")
